## Задание 5.
В онлайн-инструменте PyHealth выберите наиболее
интересующий вас пайплайн (например, Mortality Prediction)
и попробуйте его прогнать (графа Tutorials) на
предложенных разработчиками датасетах (графа Datasets)
стартовой страницы.

Выберите модель, подходящую для вашей задачи,
аргументируйте, почему именно эта модель кажется вам
наилучшей. Обучите модель и протестируйте ее, оцените
качество.

#### Установка библиотек

In [1]:
!pip install pyhealth -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00


#### Загрузка данных

In [2]:
from pyhealth.datasets import MIMIC3Dataset

mimic3_ds = MIMIC3Dataset(
        root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
)

mimic3_ds.stat()

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 79.68455648422241s
finish parsing DIAGNOSES_ICD : 52.654137134552s
finish parsing PROCEDURES_ICD : 44.52501630783081s
finish parsing PRESCRIPTIONS : 140.04596376419067s


Mapping codes: 100%|██████████| 49993/49993 [00:01<00:00, 33478.24it/s]



Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 49993
	- Number of visits: 52769
	- Number of visits per patient: 1.0555
	- Number of events per visit in DIAGNOSES_ICD: 9.1038
	- Number of events per visit in PROCEDURES_ICD: 3.2186
	- Number of events per visit in PRESCRIPTIONS: 25.9600



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 49993\n\t- Number of visits: 52769\n\t- Number of visits per patient: 1.0555\n\t- Number of events per visit in DIAGNOSES_ICD: 9.1038\n\t- Number of events per visit in PROCEDURES_ICD: 3.2186\n\t- Number of events per visit in PRESCRIPTIONS: 25.9600\n'

In [3]:
mimic3_ds.info()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



In [4]:
from pyhealth.tasks import readmission_prediction_mimic3_fn

mimic3_ds = mimic3_ds.set_task(task_fn=readmission_prediction_mimic3_fn)

mimic3_ds.stat()

Generating samples for readmission_prediction_mimic3_fn: 100%|██████████| 49993/49993 [00:00<00:00, 406260.04it/s]


Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: readmission_prediction_mimic3_fn
	- Number of samples: 2194
	- Number of patients: 2116
	- Number of visits: 2194
	- Number of visits per patient: 1.0369
	- conditions:
		- Number of conditions per sample: 10.3414
		- Number of unique conditions: 2526
		- Distribution of conditions (Top-10): [('4019', 875), ('41401', 591), ('42731', 540), ('25000', 483), ('4280', 472), ('2724', 324), ('5849', 297), ('53081', 280), ('5990', 264), ('2720', 254)]
	- procedures:
		- Number of procedures per sample: 4.0789
		- Number of unique procedures: 815
		- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 181), ('3722', 177)]
	- drugs:
		- Number of drugs per sample: 35.5228
		- Number of unique drugs: 2424
		- Distribution of drugs (Top-10): [('00338001702', 1023), ('00008084199', 903), ('51079025520', 893), ('00338004938

"Statistics of sample dataset:\n\t- Dataset: MIMIC3Dataset\n\t- Task: readmission_prediction_mimic3_fn\n\t- Number of samples: 2194\n\t- Number of patients: 2116\n\t- Number of visits: 2194\n\t- Number of visits per patient: 1.0369\n\t- conditions:\n\t\t- Number of conditions per sample: 10.3414\n\t\t- Number of unique conditions: 2526\n\t\t- Distribution of conditions (Top-10): [('4019', 875), ('41401', 591), ('42731', 540), ('25000', 483), ('4280', 472), ('2724', 324), ('5849', 297), ('53081', 280), ('5990', 264), ('2720', 254)]\n\t- procedures:\n\t\t- Number of procedures per sample: 4.0789\n\t\t- Number of unique procedures: 815\n\t\t- Distribution of procedures (Top-10): [('3893', 458), ('3961', 419), ('9904', 393), ('8856', 250), ('3615', 234), ('9604', 222), ('966', 213), ('8872', 208), ('9671', 181), ('3722', 177)]\n\t- drugs:\n\t\t- Number of drugs per sample: 35.5228\n\t\t- Number of unique drugs: 2424\n\t\t- Distribution of drugs (Top-10): [('00338001702', 1023), ('000080841

In [5]:
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader

# разделение датасета
train_dataset, val_dataset, test_dataset = split_by_patient(mimic3_ds, [0.8, 0.1, 0.1])

train_loader = get_dataloader(train_dataset, batch_size=64, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=64, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=64, shuffle=False)

### Метрики

In [30]:
all_metrics = [
        "pr_auc",
        "roc_auc",
        "accuracy",
        "balanced_accuracy"
    ]

## Модели

In [31]:
import pandas as pd

In [32]:
df_results = pd.DataFrame()

### Transformer

In [33]:
from pyhealth.models import Transformer

model = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [34]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)


Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
          )
          (i

INFO:pyhealth.trainer:Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
  

Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cuda


...

INFO:pyhealth.trainer:balanced_accuracy: 0.5000


loss: 0.4533


INFO:pyhealth.trainer:loss: 0.4533


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [35]:
from pyhealth.metrics.binary import binary_metrics_fn

y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 162.51it/s]


{'pr_auc': 0.09623722899616141,
 'roc_auc': 0.47410967882892724,
 'accuracy': 0.8918918918918919,
 'balanced_accuracy': 0.49748743718592964}

In [36]:
df_results['Transformer'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### Retain (механизм внимания)

In [37]:
from pyhealth.models import RETAIN

model = RETAIN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [38]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


INFO:pyhealth.trainer:RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cuda


...

INFO:pyhealth.trainer:loss: 0.4228


New best roc_auc score (0.5754) at epoch-4, step-140


INFO:pyhealth.trainer:New best roc_auc score (0.5754) at epoch-4, step-140


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [39]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 90.03it/s]


{'pr_auc': 0.12094169424066836,
 'roc_auc': 0.551234433034739,
 'accuracy': 0.8873873873873874,
 'balanced_accuracy': 0.4949748743718593}

In [40]:
df_results['Retain'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### RNN

In [41]:
from pyhealth.models import RNN

model = RNN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [42]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


INFO:pyhealth.trainer:RNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (rnn): ModuleDict(
    (conditions): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
    (procedures): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cuda


...

INFO:pyhealth.trainer:balanced_accuracy: 0.5000


loss: 0.4607


INFO:pyhealth.trainer:loss: 0.4607


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [43]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 149.75it/s]


{'pr_auc': 0.11138655340469839,
 'roc_auc': 0.5350666375355037,
 'accuracy': 0.8963963963963963,
 'balanced_accuracy': 0.5}

In [44]:
df_results['RNN'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### CNN

In [45]:

from pyhealth.models import CNN

model = CNN(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)

In [46]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool1d(output_size=1)
    )
    (procedures): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,

INFO:pyhealth.trainer:CNN(
  (embeddings): ModuleDict(
    (conditions): Embedding(2528, 128, padding_idx=0)
    (procedures): Embedding(817, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool1d(output_size=1)
    )
    (procedures): CNNLayer(
      (cnn): ModuleDict(
        (CNN-0): CNNBlock(
          (conv1): Sequential(
            (0): Conv1d(128

Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cuda


...

INFO:pyhealth.trainer:balanced_accuracy: 0.5388


loss: 0.5553


INFO:pyhealth.trainer:loss: 0.5553


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [47]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 199.12it/s]


{'pr_auc': 0.11295986475631542,
 'roc_auc': 0.5274197072318112,
 'accuracy': 0.8828828828828829,
 'balanced_accuracy': 0.49246231155778897}

In [48]:
df_results['CNN'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

### Deepr

In [49]:
from pyhealth.models import Deepr

model = Deepr(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="label",
    mode="binary",
)


In [50]:
trainer = Trainer(model=model, metrics=all_metrics)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor='roc_auc',
)

Deepr(
  (embeddings): ModuleDict(
    (conditions): Embedding(2529, 128, padding_idx=0)
    (procedures): Embedding(818, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
    (procedures): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


INFO:pyhealth.trainer:Deepr(
  (embeddings): ModuleDict(
    (conditions): Embedding(2529, 128, padding_idx=0)
    (procedures): Embedding(818, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (cnn): ModuleDict(
    (conditions): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
    (procedures): DeeprLayer(
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)


Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


INFO:pyhealth.trainer:Metrics: ['pr_auc', 'roc_auc', 'accuracy', 'balanced_accuracy']


Device: cuda


...

INFO:pyhealth.trainer:balanced_accuracy: 0.5000


loss: 0.4764


INFO:pyhealth.trainer:loss: 0.4764


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [51]:
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

Evaluation: 100%|██████████| 4/4 [00:00<00:00, 207.11it/s]


{'pr_auc': 0.11145315323113258,
 'roc_auc': 0.5427135678391961,
 'accuracy': 0.8918918918918919,
 'balanced_accuracy': 0.49748743718592964}

In [52]:
df_results['Deepr'] = binary_metrics_fn(y_true, y_prob, metrics=all_metrics)

## Результаты

In [53]:
df_results

,Transformer,Retain,RNN,CNN,Deepr
pr_auc,0.096237,0.120942,0.111387,0.112960,0.111453
roc_auc,0.474110,0.551234,0.535067,0.527420,0.542714
accuracy,0.891892,0.887387,0.896396,0.882883,0.891892
balanced_accuracy,0.497487,0.494975,0.500000,0.492462,0.497487


По roc-auc и pr_auc лучшая модель Retain. По accuracy - RNN. Видимо для задач такого рода хорошо подойдет модель с механизмом внимания.